### Simple Graph Convolution model using torch geometric with Cora

In [1]:
# imports
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv
from sklearn.manifold import TSNE
import torch.nn.functional as F
import matplotlib.pyplot as plt 


In [2]:
# Import the data
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]
print("Cora", data)

Coda Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [3]:
# Construct the model
SGC_model = SGConv(
    in_channels=data.num_features, # Number of features
    out_channels=dataset.num_classes, # Dimension of embedding
    K=1, # Only one iteration
    cached=True # Same computation
    )

In [4]:
# Get embedding
print(f"Shape of the original data: {data.x.shape}")
print(f"Shape of the embedding data: {SGC_model(data.x, data.edge_index).shape}")

Shape of the original data: torch.Size([2708, 1433])
Shape of the embedding data: torch.Size([2708, 7])


In [5]:
# Construct the model for classification
class SGCNet():
    def __init__(self, in_channels, out_channels):
        super(SGCNet, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.conv1 = SGConv(
            self.in_channels, self.out_channels, K=1, cached=True) 
    
    def forward(self):
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv

dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', dataset)
dataset = Planetoid(path, dataset)
data = dataset[0]


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SGConv(dataset.num_features, dataset.num_classes, K=2,
                            cached=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=0.005)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')